I have a system that stores data in a two-dimensional numpy array, with each element being one of `(-1,0,1)`

I remembered reading many years ago about the  [Balanced Ternary](https://en.wikipedia.org/wiki/Balanced_ternary) system for writing numbers, and I thought it might be fun to see if I could use it to represent  these arrays as a single integer.


In the binary system, a number is represented as a sequence of ones and zeros, each one representing a multiple of a power of 2.

In a Balanced Ternary system, a number is represented as a sequence of ones, zeros, and *minus ones*, each one representing a multiple of a power of 3.

So for example, the number eight is represented as

`1,0,-1`, 

which can be understood as 

` 1*9 + 0*3 + -1*1 = 8`

For some reason I find this system very aesthetically pleasing.  There's probably not a lot of practical use to it, although the Russians did once build an entire computer based on this system, rather than on binary.

Anyway, here's a quick python implementation of the system.

In [10]:
import pandas
import numpy as np

In [11]:
def encode(i):
    def remainders(i):
        while i:
            i,mod=divmod(i,3)
            yield mod
        yield 0
    r = list(remainders(i))
    i=0
    while i < len(r):
        if r[i]>1:
            r[i]  -=3
            r[i+1]+=1
        i+=1
    if r[-1]==0:r.pop()
    return list(reversed(r))

In [12]:
def decode(l):
    l = list(reversed(l))
    out=0
    for i in range(len(l)):
        out+=(int(l[i]))*(3**i)
    return out

In [13]:
encode(42)

[1, -1, -1, -1, 0]

So the balanced ternary representation of 42 is 1,-1,-1,-1,0

For comparison, the binary representation of 42 is:

In [14]:
list("{0:b}".format(42))

['1', '0', '1', '0', '1', '0']

## Now lets encode all the numbers up to 42

In [16]:
pandas.DataFrame({
    "encoded":[",".join(str(j) for j in encode(i)) for i in range(43) ],
})


,encoded
0,
1,1
2,"1,-1"
3,"1,0"
4,"1,1"
5,"1,-1,-1"
6,"1,-1,0"
7,"1,-1,1"
8,"1,0,-1"
9,"1,0,0"


## And finally we use this system to encode a numpy array into a single integer

In [17]:
a = np.array( [[ 1,  0,  0,  0,  1,  0, -1,  0],
               [ 0,  0,  0,  1,  0,  1,  0,  0],
               [-1,  0, -1,  0, -1,  1,  0,  1],
               [ 0,  0,  0,  0,  0,  0,  0,  0],
               [ 1,  0,  0,  1,  0,  0, -1,  1],
               [ 1,  0, -1,  0,  0,  0,  0,  1],
               [ 0,  0,  0,  0, -1,  0,  0,  1],
               [ 0,  0,  0, -1,  0,  0,  0,  0]], dtype=np.int32)

In [18]:
a

array([[ 1,  0,  0,  0,  1,  0, -1,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0],
       [-1,  0, -1,  0, -1,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  1,  0,  0, -1,  1],
       [ 1,  0, -1,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  1],
       [ 0,  0,  0, -1,  0,  0,  0,  0]], dtype=int32)

In [19]:
decode(a.flatten())

1157128765853564328061712266998

**So that entire 8x8 array can be represented as a single integer.**

Let's check that we get back our original data if we decode it:

In [20]:
np.array(encode(1157128765853564328061712266998)).reshape((8,8))

array([[ 1,  0,  0,  0,  1,  0, -1,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0],
       [-1,  0, -1,  0, -1,  1,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  1,  0,  0, -1,  1],
       [ 1,  0, -1,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  1],
       [ 0,  0,  0, -1,  0,  0,  0,  0]])